In [1]:
import fnmatch
import json
import getpass
import os
import pathlib
import datetime
                    
from dask.distributed import LocalCluster, SSHCluster 
from laserfarm import Retiler, DataProcessing, GeotiffWriter, MacroPipeline
from laserfarm.remote_utils import get_wdclient, get_info_remote, list_remote

## Global Configuration

In [2]:
# Configurations

import fnmatch
import json
import getpass
import os
import pathlib
import datetime
                    
from dask.distributed import LocalCluster, SSHCluster 
from laserfarm import Retiler, DataProcessing, GeotiffWriter, MacroPipeline
from laserfarm.remote_utils import get_wdclient, get_info_remote, list_remote

conf_remote_path_root = pathlib.Path('/webdav')
conf_remote_path_ahn = pathlib.Path('/webdav/ahn')
conf_remote_path_retiled = pathlib.Path('/webdav/retiled/')
conf_remote_path_targets = pathlib.Path('/webdav/targets')
conf_local_tmp = pathlib.Path('/tmp')

param_hostname = 'https://lifewatch.lab.uvalight.net:32443'
param_login = 'ZDFmNDg0Nzk0ZTJi'
param_password = 'NDBjMmMzODJkYTBh'


conf_wd_opts = { 'webdav_hostname': param_hostname, 'webdav_login': param_login, 'webdav_password': param_password}


param_feature_names = ['perc_95_normalized_height','mean_normalized_height', 'std_normalized_height','pulse_penetration_ratio','entropy_normalized_height','point_density']
param_feature_index = 0

## Fetching Laz Files from remote WebDAV

In [3]:
# Fetch Laz Files 23-03-22

laz_files = [f for f in list_remote(get_wdclient(conf_wd_opts), conf_remote_path_ahn.as_posix())
             if f.lower().endswith('.laz')]

## Retiling of big files into smaller tiles

In [4]:
# Retiling 23-03-22

remote_path_retiled = str(conf_remote_path_retiled)

grid_retile = {
    'min_x': -113107.81,
    'max_x': 398892.19,
    'min_y': 214783.87,
    'max_y': 726783.87,
    'n_tiles_side': 512
}


retiling_input = {
    'setup_local_fs': {'tmp_folder': conf_local_tmp.as_posix()},
    'pullremote': conf_remote_path_ahn.as_posix(),
    'set_grid': grid_retile,
    'split_and_redistribute': {},
    'validate': {},
    'pushremote': conf_remote_path_retiled.as_posix(),
    'cleanlocalfs': {}
}

file = laz_files
retiler = Retiler(file).config(retiling_input).setup_webdav_client(conf_wd_opts)
retiler_output = retiler.run()

2022-04-03 16:16:51,087 -           laserfarm.pipeline_remote_data -       INFO - Input dir set to /tmp/pipeline_input
2022-04-03 16:16:51,089 -           laserfarm.pipeline_remote_data -       INFO - Output dir set to /tmp/pipeline_output
2022-04-03 16:16:51,091 -           laserfarm.pipeline_remote_data -       INFO - Pulling from WebDAV /webdav/ahn/C_18HZ2.LAZ ...
2022-04-03 16:16:52,224 -           laserfarm.pipeline_remote_data -       INFO - ... pulling completed.
2022-04-03 16:16:52,225 -                        laserfarm.retiler -       INFO - Setting up the target grid
2022-04-03 16:16:52,228 -                        laserfarm.retiler -       INFO - Splitting file /tmp/pipeline_input/C_18HZ2.LAZ with PDAL ...
2022-04-03 16:16:52,732 -                        laserfarm.retiler -       INFO - ... splitting completed.
2022-04-03 16:16:52,735 -                        laserfarm.retiler -       INFO - Redistributing files to tiles ...
2022-04-03 16:16:52,793 -                        l

## Fetching retilied files (tiles) from remote WebDAV

In [6]:
# Fetch Tiles 1-04-22
remote_path_retiled

tiles = [t.strip('/') for t in list_remote(get_wdclient(conf_wd_opts), conf_remote_path_retiled.as_posix())
         if fnmatch.fnmatch(t, 'tile_*_*/')]

## Extract features from tiles

In [8]:
# Feature Extraction 23-03-22

t = tiles[0]

tile_mesh_size = 10.
features = ['perc_95_normalized_height']

grid_feature = {
    'min_x': -113107.81,
    'max_x': 398892.19,
    'min_y': 214783.87,
    'max_y': 726783.87,
    'n_tiles_side': 512
}

feature_extraction_input = {
    'setup_local_fs': {'tmp_folder': conf_local_tmp.as_posix()},
    'pullremote': conf_remote_path_retiled.as_posix(),
    'load': {'attributes': ['raw_classification']},
    'normalize': 1,
    'apply_filter': {
        'filter_type': 'select_equal', 
        'attribute': 'raw_classification',
        'value': [1, 6]#ground surface (2), water (9), buildings (6), artificial objects (26), vegetation (?), and unclassified (1)
    },
    'generate_targets': {
        'tile_mesh_size' : tile_mesh_size,
        'validate' : True,
        **grid_feature
    },
    'extract_features': {
        'feature_names': features,
        'volume_type': 'cell',
        'volume_size': tile_mesh_size
    },
    'export_targets': {
        'attributes': features,
        'multi_band_files': False
    },
    'pushremote': conf_remote_path_targets.as_posix(),
#     'cleanlocalfs': {}
}
idx = (t.split('_')[1:])
processing = DataProcessing(t, tile_index=idx).config(feature_extraction_input).setup_webdav_client(conf_wd_opts)
processing.run()

2022-04-03 16:17:42,704 -           laserfarm.pipeline_remote_data -       INFO - Input dir set to /tmp/pipeline_input
2022-04-03 16:17:42,705 -           laserfarm.pipeline_remote_data -       INFO - Output dir set to /tmp/pipeline_output
2022-04-03 16:17:42,707 -           laserfarm.pipeline_remote_data -       INFO - Pulling from WebDAV /webdav/retiled/tile_213_285 ...
2022-04-03 16:17:44,604 -           laserfarm.pipeline_remote_data -       INFO - ... pulling completed.
2022-04-03 16:17:44,605 -                laserfarm.data_processing -       INFO - Loading point cloud data ...
2022-04-03 16:17:44,606 -                laserfarm.data_processing -       INFO - ... loading /tmp/pipeline_input/tile_213_285/C_18HZ2_1.LAZ
2022-04-03 16:17:44,609 -                          pylas.lasreader -      ERROR - lazrs failed to decompress points: lazrs is not installed
2022-04-03 16:17:45,043 -                laserfarm.data_processing -       INFO - ... loading completed.
2022-04-03 16:17:45,044

In [10]:
# GeoTIFF Export 23-03-22

feature = features[0]
remote_path_geotiffs = conf_remote_path_ahn.parent / 'geotiffs'

# setup input dictionary to configure the GeoTIFF export pipeline
geotiff_export_input = {
    'setup_local_fs': {'tmp_folder': conf_local_tmp.as_posix()},
    'pullremote': conf_remote_path_targets.as_posix(),
    'parse_point_cloud': {},
    'data_split': {'xSub': 1, 'ySub': 1},
    'create_subregion_geotiffs': {'output_handle': 'geotiff'},
    'pushremote': remote_path_geotiffs.as_posix(),
    'cleanlocalfs': {}   
}

writer = GeotiffWriter(input_dir=feature, bands=feature).config(geotiff_export_input).setup_webdav_client(conf_wd_opts)
writer.run()

2022-04-03 16:18:02,469 -           laserfarm.pipeline_remote_data -       INFO - Input dir set to /tmp/pipeline_input
2022-04-03 16:18:02,471 -           laserfarm.pipeline_remote_data -       INFO - Output dir set to /tmp/pipeline_output
2022-04-03 16:18:02,473 -           laserfarm.pipeline_remote_data -       INFO - Pulling from WebDAV /webdav/targets/perc_95_normalized_height ...
2022-04-03 16:18:04,276 -           laserfarm.pipeline_remote_data -       INFO - ... pulling completed.
2022-04-03 16:18:04,278 -                 laserfarm.geotiff_writer -       INFO - 1 PLY files found
2022-04-03 16:18:04,370 -                 laserfarm.geotiff_writer -       INFO - No. of points per file: 10000
2022-04-03 16:18:04,372 -                 laserfarm.geotiff_writer -       INFO - Resolution: (10.0m x 10.0m)
2022-04-03 16:18:04,373 -                 laserfarm.geotiff_writer -       INFO - Splitting data into (1x1) sub-regions
2022-04-03 16:18:04,374 -                 laserfarm.geotiff_write